# Level 2: Rice Crop Yield Forecasting Tool Benchmark Notebook

In [3]:
%pip install tsmoothie
%pip install catboost

  Using cached tsmoothie-1.0.4-py3-none-any.whl (21 kB)
  Using cached simdkalman-1.0.2-py2.py3-none-any.whl (11 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached catboost-1.1.1-cp310-none-manylinux1_x86_64.whl (76.6 MB)
  Using cached plotly-5.14.0-py2.py3-none-any.whl (15.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
from ipyleaflet import Map, Marker
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
from tsmoothie.utils_func import sim_randomwalk
from tsmoothie.smoother import LowessSmoother, KalmanSmoother
from tsmoothie.bootstrap import BootstrappingWrapper
import xarray as xr

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor, Pool

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
pc.settings.set_subscription_key('')

# Others
import requests
import rich.table
from datetime import datetime
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()

## Response Variable

Before building the model, we need to load in the rice crop yield data. In particular, rice crop yield data was collected for the period of late-2021 to mid-2022 over the Chau Phu, Chau Thanh and Thoai Son districts.

This is a dense rice crop region with a mix of double and triple cropping cycles.For this demonstration, we have assumed a triple cropping (3 cycles per year) for all the data points, but you are free to explore the impact of cropping cycles on the yield.You will have to map every data point with its corresponding crop cycle.
The crop cycles are Winter-Spring ( November – April) and the Summer-Autumn (April – August). E.g., the harvest date for the first entry is 15th July 2022. The corresponding crop cycle will be Summer-Autumn (April – August). 

The data consists of geo locations (Latitude and Longitude), District, Season, Rice Crop Intensity, Date of Harvest, Field Size (in Hectares) with the yield in each geo location.

In [8]:
crop_yield_data = pd.read_csv("PlanetaryComputerExamples/OpenDataChallenge/Data/Crop_Yield_Data_challenge_2.csv")
crop_yield_data.head()

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,15-07-2022,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,15-07-2022,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,15-07-2022,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,15-07-2022,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,14-07-2022,3.30,6400


## Predictor Variables

### Accessing the Sentinel-1 Data

<p align = "Justify">To get the Sentinel-1 data, we write a function called <i><b>get_sentinel_data.</b></i> This function will fetch VV, VH band values and VV/VH values for a particular location over the specified time window. In this example, we have taken the VV, VH, and VV/VH values for 4 months in each season.</p>

In [5]:
def create_bbox(lat_long, size_px):
    
    box_size_deg = 0.00007 * size_px 

    min_lon = lat_long[1]-box_size_deg/2
    min_lat = lat_long[0]-box_size_deg/2
    max_lon = lat_long[1]+box_size_deg/2
    max_lat = lat_long[0]+box_size_deg/2

    bbox = (min_lon, min_lat, max_lon, max_lat)
    return bbox

def smooth_data(data, window_size=3):
    data = data.rolling(window_size, center=True).mean()
    return data

def db_scale(x):
    return 10 * np.log10(x)

#function to get date from string 6 months prior
def get_date(date_string):
    date = datetime.strptime(date_string, '%d-%m-%Y')
    datePrior = date - pd.DateOffset(months=6)
    date = datetime.strftime(date, '%Y-%m-%d')
    datePrior = datetime.strftime(datePrior, '%Y-%m-%d')
    #get delta between date and date_string
    return str(datePrior + '/' + date)

In [6]:
def get_sentinel_data(longitude, latitude, harvest_date,assets):
    
    '''
    Returns a list of VV,VH, VV/VH values for a given latitude and longitude over a given time period (based on the season)
    Attributes:
    longitude - Longitude
    latitude - Latitude
    season - The season for which band values need to be extracted.
    assets - A list of bands to be extracted
    
    '''
    
    time_slice = get_date(harvest_date)
        
    

    
    bbox_of_interest = create_bbox((latitude, longitude), 3)
    time_of_interest = str(time_slice)
    
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search = catalog.search(
        collections=["sentinel-1-rtc"], 
        bbox=bbox_of_interest, 
        datetime=time_of_interest
    )
    items = list(search.get_all_items())
    
    data = stac_load(items,bands = assets, patch_url=pc.sign, bbox=bbox_of_interest)

    
              
    return data

In [10]:
## Get Sentinel-1-RTC Data

assets = ['vh','vv']
longitude = crop_yield_data['Longitude'].astype(float).values
latitude = crop_yield_data['Latitude'].astype(float).values
harvest_date = crop_yield_data['Date of Harvest'].values
datasets = []
for i in tqdm(range(550,len(longitude))):
    data = db_scale(get_sentinel_data(longitude[i],latitude[i], harvest_date[i], assets))
    data.to_netcdf('PlanetaryComputerExamples/OpenDataChallenge/Data/RawData_2/raw_data_' + str(i) + '.nc')
    datasets.append(data)

100%|██████████| 7/7 [00:55<00:00,  7.94s/it]


## FEATURE BUILDING


In [43]:
#get sample data from Data/RawData files
dataArray = []
for i in tqdm(range(0,len(crop_yield_data))):
    data = xr.open_dataset('PlanetaryComputerExamples/OpenDataChallenge/Data/RawData_2/raw_data_'+str(i)+'.nc')
    data = data.mean(dim=['x','y']).compute()
    data = data.assign_coords({'time':np.arange(0,len(data.time))})
    dataArray.append(data)


100%|██████████| 557/557 [00:05<00:00, 100.72it/s]


In [44]:
data = xr.concat(dataArray, dim='place')
#drop last 3 time steps
data = data.isel(time=slice(0,-3))

In [11]:
#interpolate na
data = data.interpolate_na(dim='time', method='linear')

In [12]:
#add cross-co polarisation
crossCo = (data.vh/data.vv).to_dataset(name='crossCo')
data = data.merge(crossCo)

In [13]:
#apply time series smoothing
smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
smooth_data = []	
for i in tqdm(range(0,len(data.place))):
    smoother.smooth(data.isel(place=i).to_array(dim='time'))
    smooth_data.append(xr.DataArray(smoother.smooth_data, dims=['bands', 'scene'], coords={'bands':['vv','vh','crossCo'], 'scene':range(0, 35)}))


smooth_data = xr.concat(smooth_data, dim='place')

100%|██████████| 557/557 [00:01<00:00, 500.57it/s]


In [14]:
crop_data = smooth_data.stack(variables=('bands', 'scene')).to_numpy()

In [145]:
crop_data

array([[-18.923464  , -17.957426  , -16.986423  , ...,   1.6790948 ,
          1.6937134 ,   1.7080252 ],
       [-12.492396  , -13.942531  , -15.295291  , ...,   0.5891562 ,
          0.53165424,   0.47630247],
       [-18.472778  , -18.03032   , -17.503496  , ...,   1.654949  ,
          1.7303727 ,   1.8078941 ],
       ...,
       [-15.690487  , -16.062609  , -16.279774  , ...,   1.6024942 ,
          1.7554646 ,   1.953673  ],
       [ -9.587777  , -11.550083  , -13.366557  , ...,   2.0698874 ,
          2.0250323 ,   1.9860201 ],
       [-17.798119  , -15.748918  , -13.802306  , ...,   1.8359718 ,
          1.6063745 ,   1.3026239 ]], dtype=float32)

In [15]:
crop_data.shape

(557, 105)

### Train and Test Split 

<p align="justify">We will now split the data into 80% training data and 20% test data. Scikit-learn alias “sklearn” is a robust library for machine learning in Python. The scikit-learn library has a <i><b>model_selection</b></i> module in which there is a splitting function <i><b>train_test_split</b></i>. You can use the same.</p>

In [16]:
X = crop_data
y = crop_yield_data ['Rice Yield (kg/ha)'].values
# Choose any random state
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=21)

### Model Training

In [18]:
#prepares catboost data

train_data = X_train

train_labels = y_train

#split data into train and test

test_data = catboost_pool = Pool(data=X_test, label=y_test)

model = CatBoostRegressor(  
                           n_estimators=500,
                           learning_rate=0.1, 
                           depth=3, 
                           loss_function='RMSE', 
                           has_time=True
                           )


model.fit(train_data, train_labels)


0:	learn: 753.1336137	total: 51.9ms	remaining: 25.9s
1:	learn: 719.3132756	total: 54.2ms	remaining: 13.5s
2:	learn: 685.3056817	total: 56ms	remaining: 9.28s
3:	learn: 658.2116241	total: 57.9ms	remaining: 7.18s
4:	learn: 632.2435129	total: 59.8ms	remaining: 5.92s
5:	learn: 608.5207433	total: 61.5ms	remaining: 5.06s
6:	learn: 590.8647127	total: 63.2ms	remaining: 4.45s
7:	learn: 573.1513133	total: 65ms	remaining: 4s
8:	learn: 555.4294176	total: 67.1ms	remaining: 3.66s
9:	learn: 543.1449652	total: 69.1ms	remaining: 3.38s
10:	learn: 530.1204071	total: 71.1ms	remaining: 3.16s
11:	learn: 520.0739474	total: 72.8ms	remaining: 2.96s
12:	learn: 512.6060437	total: 74.7ms	remaining: 2.8s
13:	learn: 506.8519475	total: 76.6ms	remaining: 2.66s
14:	learn: 499.4465259	total: 78.5ms	remaining: 2.54s
15:	learn: 492.2639704	total: 80.4ms	remaining: 2.43s
16:	learn: 487.9054707	total: 82.2ms	remaining: 2.33s
17:	learn: 482.4673038	total: 84ms	remaining: 2.25s
18:	learn: 478.2795193	total: 86ms	remaining: 2.

In [ ]:
grid = {'n_estimators': [500, 1000, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

model.grid_search(grid, train_data, train_labels)

## Model Evaluation

In [19]:
pred = model.predict(X_test)
r2 = r2_score(y_test, pred)
print('Testing performance')
print('R2: {:.2f}'.format(r2))

Testing performance
R2: 0.51


### Out-Sample Evaluation

## Submission

Once you are happy with your model, you can make a submission. To make a submission, you will need to use your model to make the yield predictions of rice crop for a set of test coordinates we have provided in the <a href="https://challenge.ey.com/api/v1/storage/admin-files/8515054086281302-63ca8f827b1fe300146c7e21-challenge_2_submission_template.csv"><b>"challenge_2_submission_template.csv"</b></a> file and upload the file onto the challenge platform.

In [24]:
test_file = pd.read_csv('PlanetaryComputerExamples/OpenDataChallenge/Data/Challenge_2_submission_template.csv')
test_file.head()

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,10-04-2022,1.40,NaN
1,2,Chau_Thanh,10.400189,105.331053,SA,T,15-07-2022,1.32,NaN
2,3,Chau_Phu,10.505489,105.203926,SA,D,14-07-2022,1.40,NaN
3,4,Chau_Phu,10.523520,105.138274,WS,D,10-04-2022,1.80,NaN
4,5,Thoai_Son,10.294660,105.248528,SA,T,20-07-2022,2.20,NaN


In [25]:
## Get Sentinel-1-RTC Data

assets = ['vh','vv']
longitude = test_file['Longitude'].astype(float).values
latitude = test_file['Latitude'].astype(float).values
harvest_date = test_file['Date of Harvest'].values
datasets = []
for i in tqdm(range(0,len(longitude))):
    data = db_scale(get_sentinel_data(longitude[i],latitude[i], harvest_date[i], assets))
    data.to_netcdf('PlanetaryComputerExamples/OpenDataChallenge/Data/SubmissionData_2/raw_data_' + str(i) + '.nc')
    datasets.append(data)

100%|██████████| 100/100 [10:46<00:00,  6.47s/it]


In [33]:
#get sample data from Data/RawData files
dataArray = []
for i in tqdm(range(0,len(test_file))):
    data = xr.open_dataset('PlanetaryComputerExamples/OpenDataChallenge/Data/SubmissionData_2/raw_data_'+str(i)+'.nc')
    data = data.mean(dim=['x','y']).compute()
    data = data.assign_coords({'time':np.arange(0,len(data.time))})
    dataArray.append(data)

100%|██████████| 100/100 [00:01<00:00, 96.36it/s]


In [34]:
data = xr.concat(dataArray, dim='place')
#drop last 3 time steps
data = data.isel(time=slice(0,-3))

In [35]:
#interpolate na
data = data.interpolate_na(dim='time', method='linear', fill_value='extrapolate')

In [36]:
#add cross-co polarisation
crossCo = (data.vh/data.vv).to_dataset(name='crossCo')
data = data.merge(crossCo)

In [37]:
#apply time series smoothing
smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
smooth_data = []	
for i in tqdm(range(0,len(data.place))):
    smoother.smooth(data.isel(place=i).to_array(dim='time'))
    smooth_data.append(xr.DataArray(smoother.smooth_data, dims=['bands', 'scene'], coords={'bands':['vv','vh','crossCo'], 'scene':range(0, 35)}))


smooth_data = xr.concat(smooth_data, dim='place')

100%|██████████| 100/100 [00:00<00:00, 530.52it/s]


In [38]:
crop_data = smooth_data.stack(variables=('bands', 'scene')).to_numpy()

In [39]:
pred = model.predict(crop_data)

In [40]:
#Combining the results into dataframe
test_file['Predicted Rice Yield (kg/ha)'] = list(pred)

In [42]:
#Dumping the predictions into a csv file.
test_file.to_csv("PlanetaryComputerExamples/OpenDataChallenge/Data/Submission/challenge_2_submission_rice_crop_yield_prediction.csv",index = False)